In [1]:
%load_ext autoreload
%autoreload 2

import pathlib
import os

import flax
from flax import jax_utils
import jax
import jax.numpy as jnp
import numpy as np
import optax
import torch
import tensorflow as tf
import tqdm

import init2winit
import fastmri

import i2w

from fastmri.models import unet as t_unet
from fastmri.pl_modules import data_module
from fastmri.pl_modules import unet_module
from fastmri.data.transforms import UnetDataTransform
from fastmri.data.subsample import create_mask_for_mask_type

from init2winit.model_lib import unet as f_unet
from init2winit.dataset_lib import fastmri_dataset
from init2winit.dataset_lib import data_utils
from init2winit.optimizer_lib import optimizers

jax.devices()

[GpuDevice(id=0, process_index=0),
 GpuDevice(id=1, process_index=0),
 GpuDevice(id=2, process_index=0),
 GpuDevice(id=3, process_index=0),
 GpuDevice(id=4, process_index=0),
 GpuDevice(id=5, process_index=0),
 GpuDevice(id=6, process_index=0),
 GpuDevice(id=7, process_index=0)]

# ConvBlock

In [2]:
t_data = np.random.uniform(size=(8, 1, 4, 5))
f_data = np.transpose(t_data, axes=(0, 2, 3, 1))

## `fastmri`

In [3]:
t_block = t_unet.ConvBlock(in_chans=1, out_chans=1, drop_prob=0.0)
t_out = t_block(torch.Tensor(t_data)).detach().numpy()

for name, param in t_block.named_parameters():
     print(name, param.data.numpy().shape)
        
t_params = list(t_block.parameters())

layers.0.weight (1, 1, 3, 3)
layers.4.weight (1, 1, 3, 3)


## `init2winit`

In [4]:
f_block = f_unet.ConvBlock(out_chans=1, drop_prob=0.0)
f_block_params = f_block.init(jax.random.PRNGKey(0), f_data)

f_block_params = flax.core.unfreeze(f_block_params)
f_block_params['params']['Conv_0']['kernel'] = jnp.array(np.transpose(t_params[0].detach().numpy(), axes = (2, 3, 1, 0)))
f_block_params['params']['Conv_1']['kernel'] = jnp.array(np.transpose(t_params[1].detach().numpy(), axes = (2, 3, 1, 0)))

f_out = np.array(f_block.apply(f_block_params, f_data).transpose((0, 3, 1, 2)))

## Compared

In [5]:
np.mean(np.abs(t_out - f_out))

6.4534106e-08